In [0]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

### Mount Google Drive

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
!ls /content/drive

'My Drive'


In [46]:
!ls '/content/drive/My Drive/Tertiary Courses/Colab/DeepLearningTensorflow/dogs_cats'

test  train  validation


In [0]:
data_dir = "/content/drive/My Drive/Tertiary Courses/Colab/DeepLearningTensorflow/dogs_cats"
train_data_dir = data_dir + "/train"
val_data_dir = data_dir + "/validation"
img_width, img_height = 150, 150 

### Step 1: Data Augmentation

In [0]:
# Add augmentation configuration for the data generator of train data only
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

val_datagen = ImageDataGenerator(
    rescale = 1. / 255)

In [44]:
train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=[img_width, img_height],
    class_mode='binary',
    shuffle = True,
    batch_size = 50)

val_generator = val_datagen.flow_from_directory(
    directory = val_data_dir,
    target_size = [img_width, img_height],
    class_mode = 'binary',
    shuffle = True,
    batch_size = 50)

Found 4000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


### Step 2: Pre-Trained Base Model

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (img_width, img_height, 3))

In [15]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

### Step 3: Replace softmax Layer and add one dense layer

In [0]:
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x) #new FC layer, random init
x = Dropout(0.4)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [49]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [0]:
# Setup trainable layer
for layer in base_model.layers:
    layer.trainable = False

In [51]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [54]:
print("{:<10} Pretrained model layers: {}".format('[INFO]', len(base_model.layers)))
print("{:<10} Total number of layers : {}".format('[INFO]', len(model.layers)))

[INFO]     Pretrained model layers: 19
[INFO]     Total number of layers : 23


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [58]:
# train the model on the new data for a few epochs
import time
train_start_time = time.time()

from keras.callbacks import EarlyStopping

model.fit_generator(
    train_generator,
    epochs = 25,
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)],
    validation_data = val_generator)
model.save(data_dir + '/model/model_tf.h5')

print("It takes {:.2f} min to train the model".format((time.time() - train_start_time)/60 ))

Epoch 1/25
80/80 [==============================] - 2242s 28s/step - loss: 0.5957 - acc: 0.7043 - val_loss: 0.3703 - val_acc: 0.8215
Epoch 2/25
80/80 [==============================] - 49s 609ms/step - loss: 0.4730 - acc: 0.7655 - val_loss: 0.3040 - val_acc: 0.8625
Epoch 3/25
80/80 [==============================] - 49s 615ms/step - loss: 0.4165 - acc: 0.8040 - val_loss: 0.2819 - val_acc: 0.8730
Epoch 4/25
80/80 [==============================] - 48s 605ms/step - loss: 0.4074 - acc: 0.8157 - val_loss: 0.2907 - val_acc: 0.8785
Epoch 5/25
80/80 [==============================] - 49s 607ms/step - loss: 0.3864 - acc: 0.8207 - val_loss: 0.2955 - val_acc: 0.8740
It takes 40.62 min to train the model


### Evaluation model

In [0]:
from keras.models import load_model
model_tf = load_model(data_dir + '/model/model_tf.h5')

In [61]:
# Set up data generator for test data
from keras.preprocessing.image import ImageDataGenerator
datagen_test = ImageDataGenerator(rescale=1. / 255)

test_data = datagen_test.flow_from_directory (
    directory = data_dir + '/test',
    target_size = [img_width, img_height],
    class_mode = None,
    shuffle = False,
    batch_size = 100)

Found 200 images belonging to 2 classes.


In [0]:
# Make prediction and Check model performance
# Use model to yield score prediction for test data
scores = model_tf.predict_generator(test_data)
scores

In [63]:
# Process scores to get prediction result
y_pred = [round(score[0]) for score in scores]
y_pred

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [0]:
# Prepare actual result using filenames
y_true = [0 if 'cat' in filename else 1 for filename in test_data.filenames]

In [65]:
# Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.93


In [66]:
# Generate a report
import pandas as pd
pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames = ['True'], colnames = ['Pred'], margins = True)

Pred,0.0,1.0,All
True,,,
0,90,10,100
1,4,96,100
All,94,106,200


#### at this point, the top layers are well trained and we can start fine-tuning convolutional layers from base model. 
#### We will freeze the bottom N layers and train the remaining top layers

### Step 4: Unfreeze Layers

In [0]:
# Unfreeze and train the top 5 layers
for layer in model.layers[:5]:
    layer.trainable = False
for layer in model.layers[5:]:
    layer.trainable = True

In [0]:
model.summary()

In [0]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [69]:
import time
train_start_time = time.time()
from keras.callbacks import EarlyStopping

model.fit_generator(
    train_generator,
    epochs = 25,
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)],
    validation_data = val_generator)

model.save(data_dir + '/model/model_finetune.h5')

print("It takes {:.2f} min to train the model".format((time.time() - train_start_time)/60 ))

Epoch 1/25
80/80 [==============================] - 64s 799ms/step - loss: 0.2868 - acc: 0.8765 - val_loss: 0.2536 - val_acc: 0.9180
Epoch 2/25
80/80 [==============================] - 58s 722ms/step - loss: 0.2468 - acc: 0.8967 - val_loss: 0.1772 - val_acc: 0.9340
Epoch 3/25
80/80 [==============================] - 58s 720ms/step - loss: 0.2041 - acc: 0.9127 - val_loss: 0.2240 - val_acc: 0.9255
Epoch 4/25
80/80 [==============================] - 58s 722ms/step - loss: 0.1987 - acc: 0.9242 - val_loss: 0.1719 - val_acc: 0.9360
Epoch 5/25
80/80 [==============================] - 58s 724ms/step - loss: 0.1810 - acc: 0.9342 - val_loss: 0.1255 - val_acc: 0.9535
Epoch 6/25
80/80 [==============================] - 58s 719ms/step - loss: 0.1623 - acc: 0.9385 - val_loss: 0.1212 - val_acc: 0.9565
Epoch 7/25
80/80 [==============================] - 58s 720ms/step - loss: 0.1517 - acc: 0.9388 - val_loss: 0.1375 - val_acc: 0.9555
Epoch 8/25
80/80 [==============================] - 58s 720ms/step - 

### Load Model

In [0]:
from keras.models import load_model
model_ft = load_model(data_dir + '/model/model_finetune.h5')

In [71]:
# Make prediction and Check model performance

# Use model to yield score prediction for test data
scores = model_ft.predict_generator(test_data)
scores

array([[6.1899424e-05],
       [7.7238679e-04],
       [2.0861626e-06],
       [4.2259693e-05],
       [2.1446943e-03],
       [1.1652708e-05],
       [8.5532665e-06],
       [1.1175871e-04],
       [2.2500753e-05],
       [1.2730062e-03],
       [6.3672066e-03],
       [3.7158448e-01],
       [4.1723251e-07],
       [3.6066771e-04],
       [9.2387199e-07],
       [2.0429790e-03],
       [4.4648051e-03],
       [7.3522329e-05],
       [4.7683716e-07],
       [1.9073486e-06],
       [1.7881393e-06],
       [1.0710955e-04],
       [7.1940720e-03],
       [3.6875337e-02],
       [3.5375357e-05],
       [2.1874905e-05],
       [7.1933866e-03],
       [0.0000000e+00],
       [1.3020933e-03],
       [3.5563707e-03],
       [1.3444364e-02],
       [2.0951033e-05],
       [7.7486038e-07],
       [9.0968609e-04],
       [5.3644180e-07],
       [6.9737434e-06],
       [2.6822090e-07],
       [3.6984682e-05],
       [6.7383051e-05],
       [9.2387199e-07],
       [3.7044287e-05],
       [5.662441

In [72]:
# Process scores to get prediction result
y_pred = [round(score[0]) for score in scores]
y_pred

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [0]:
# Prepare actual result using filenames
y_true = [0 if 'cat' in filename else 1 for filename in test_data.filenames]

In [74]:
# Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.955


In [75]:
# Generate a report
import pandas as pd
pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames = ['True'], colnames = ['Pred'], margins = True)

Pred,0.0,1.0,All
True,,,
0,97,3,100
1,6,94,100
All,103,97,200


### Read image, Preprocess image, and Make prediction

In [0]:
# Define a function for reading image from url
import requests, io
from PIL import Image
def read_image_from_url(url):
    try:
        r = requests.get(url, timeout=15)
        img = Image.open(io.BytesIO(r.content))
        return img
    except:
        print("{:<10} Cannot find image from {}".format('[ERROR]', url))
        exit(1)

In [0]:
# Define a function for preprocessing image
from PIL import Image
import numpy as np
def preprocess_image(img, target_size):
    from keras.preprocessing.image import img_to_array
    img = img.resize(target_size,Image.ANTIALIAS)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32')
    img /= 255
    return img

In [0]:
def process_result(score):
  return 'dog' if score > 0.5 else 'cat'

In [81]:
# Read image, Preprocess image, and Make prediction
image = read_image_from_url('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTcc_uicIa8wyfQe4a1obrQ7mtkm7jlcfVOLqfoNrd1m7x5SQo8') # replace with any image url
image = preprocess_image(image, (img_width, img_height))
score = model_ft.predict(image)
print('Score: ' + str(score))
print('Class: ' + process_result(score))

Score: [[0.18564071]]
Class: cat
